In [4]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [5]:
# parameters
customer_id = '5'
formatted_attribute = 'capacity_c'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Multiple Buckets"""

regex_pattern=r'(?i)(cord\s?measures\s?\d*\'\s?L?)|(cord\s?measures\s?\d*.?inch)|(cord\s?measures\s?\d*.?foot)|(cord\s?measures\s?\d*.?.?\d*)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [8]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
df

product_id external_id  \
0      817156195    16741698   
1      817360018    67617845   
2      817350102    67163809   
3      988767801    69636992   
4      817192677    63186956   
...          ...         ...   
8444   817501119    43659685   
8445   817499262    43064793   
8446   817378706    42865445   
8447   817156392    16946559   
8448   817216716    41499085   

                                                         product_name  \
0                                       Wedgwood® English Lace Teapot   
1                                           Wedgwood® Parkland Teapot   
2               Classic Touch Gold Bead Apple Candy Dishes (Set of 2)   
3                Mr. Christmas® Nostalgic Lighted Cookie Jar in White   
4             NFL Dallas Cowboys 81-Piece Complete Tailgate Party Kit   
...                                                               ...   
8444       Philips Avent 10-Pack 6 Oz. Breastmilk Storage Starter Set   
8445       Tervis® NBA Charlotte Hornets 16 oz. Wrap Tumbler with Lid   
8446                                   Fiesta® Large Pitcher in White   
8447                                 Fiesta® Large Pitcher in Scarlet   
8448  Tervis® Carolina Panthers 24 oz. Colossal Wrap Tumbler with Lid   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                   Wedgwood's exquisite English Lace Teapot features an intricate, beautifully embossed lace motif inspired by traditional embroidery. Augmented with a simple platinum band, this sophisticated pattern has a subtle, timeless elegance. The subtle sophistication and graceful beauty of Wedgwood's English Lace Teapot will be treasured for generations Wonderfully intricate yet delicate embossed lace motif inspired by the tradition of lace making Simple, timeless platinum band Bone china Dishwasher safe 36 oz. capacity Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Wedgwood's striking Parkland Dinnerware Collection draws inspiration from England's elegant Palladian estates. Beautifully crafted in fine bone china, this sophisticated dinnerware creates a refined tablescape. Inspired by elegant stately homes and Palladian architecture Made in England Fine bone china Microwave and dishwasher safe 28.8 oz. capacity Measures 5.7\" H Imported   
2                                              

In [24]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [26]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df.head(1)

product_id external_id  \
0      923711    47294783   

                                                               product_name  \
0  Feiss® Federal 16.5-Inch Post-Mount Outdoor Light in Dark-Weathered Zinc   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0  With the Feiss Federal Post-Mount Wall Light you can add beauty and illumination to any outdoor area. The fixture is elegant, stately, and features a vintage design that is certain to be eye-catching, while its clear-seeded glass cover helps to hide dirt. With the Feiss Federal Post-Mount Outdoor Light you can add beauty and illumination to any outdoor space Clear seeded glass to diffuse light and hide dirt Use to light outdoor entryways, walkways, and pathways Uses 1 medium A19 bulb (not included), Mounts easily to posts Direct wire fixture, instructions included for ease of installation Metal construction Measures 16.5\" H x 7\" W Measures 19.8\" H x 7\" W Measures 25.58\" H x 7\" W Measures 35.18\" H x 7\" W ETL listed for wet locations 1-year manufacturer's warranty Imported Model OL10600DWZ 16.5-Inch; OL10601DWZ 20-Inch; OL10602DWZ 26-Inch; OL10603DWZ 35-Inch   

   customer_id attributes                    buckets values matches  
0            5       None  Sconces & Vanity Lighting   None      []

In [28]:
df=df[df['matches'].astype(str)!='[]']
df.head(1)

product_id external_id                              product_name  \
7   817680287    69534992  Conair® Knot Dr. Hot Air Brush in Purple   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
7  The Knot Dr. Hot Air Brush by Conair makes an excellent styling tool for all ages and hair types. With exceptional hair-detangling technology and 176 Flexalite bristles, this innovative brush allows you to dry and style your hair in one easy step. Dry and style your hair in one easy step KNOTECTOMY made simple has 176 Flexalite bristles Swivel cord Brush can be used on wet or dry hair 1,000 watts of power for quick drying For fine, medium, and coarse hair Tourmaline technology Wipe clean Measures 15\" L x 3\" W x 3\" H Cord measures 6' long Weighs 0.97 lb. 3-year limited manufacturer's warranty Imported Model BC116C   

   customer_id attributes             buckets values               matches  
7            5       None  Hair Styling Tools   None  [Cord measures 6' l]

In [10]:
# df[df['external_id'].astype(str)=='69600196']

In [31]:
df['matches'].explode().value_counts()

Cord measures 8' L     356
Cord measures 6' L     328
Cord measures 96\"     206
Cord measures 72\"     180
Cord measures 6'        97
                      ... 
Cord measures 35' L      1
Cord measures 5.9        1
cord Measures 6.75       1
Cord measures 9\"        1
Cord measures 138\"      1
Name: matches, Length: 176, dtype: int64

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
#     data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [ ]:
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

In [ ]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

In [ ]:
df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df

In [ ]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

In [ ]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [ ]:
df

In [ ]:
matches=df
matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches

In [ ]:
matches[curation_col] = matches['Max'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)